In [1]:
import uproot
import pandas as pd
import numpy as np

def load_mc(filename="mcdata_taue2.root", step=1):
    f = uproot.open(filename)
    mc = f['Data'].pandas.df(["Event_id", "ele_P", "BT_X", "BT_Y",
                              "BT_Z","BT_SX", "BT_SY","ele_x", 
                              "ele_y", "ele_z", "ele_sx", "ele_sy", "chisquare", ], flatten=False)
    pmc = pd.DataFrame(mc)
    pmc['numtracks'] = pmc.BT_X.apply(lambda x: len(x))
    # cuts
    shapechange = [pmc.shape[0]]
    pmc = pmc[pmc.ele_P > 0.1]
    shapechange.append(pmc.shape[0])

    pmc = pmc[pmc.ele_z < 0]
    shapechange.append(pmc.shape[0])

    pmc = pmc[pmc.numtracks > 3]
    shapechange.append(pmc.shape[0])
    print("numtracks reduction by cuts: ", shapechange)
    pmc['m_BT_X'] = pmc.BT_X.apply(lambda x: x.mean())
    pmc['m_BT_Y'] = pmc.BT_Y.apply(lambda x: x.mean())
    pmc['m_BT_Z'] = pmc.BT_Z.apply(lambda x: x.mean())

    print("len(pmc): {len}".format(len=len(pmc)))
    return pmc

In [2]:
pmc = load_mc(filename='./data/mcdata_taue2.root', step=1)

numtracks reduction by cuts:  [18724, 18679, 9616, 9106]
len(pmc): 9106


In [3]:
from tqdm import tqdm
def normalize_showers(pmc):
    showers = []
    for idx in tqdm(pmc.index):
        shower = pmc.loc[idx]
        shower_normalized = {
            'SX': shower['BT_X'] - shower['ele_x'],
            'SY': shower['BT_Y'] - shower['ele_y'],
            'SZ': shower['BT_Z'] - shower['ele_z'],
            'TX': np.tan(np.arctan(shower['BT_SX']) - np.arctan(shower['ele_sx'])),
            'TY': np.tan(np.arctan(shower['BT_SY']) - np.arctan(shower['ele_sy'])),
            'ele_P': [shower['ele_P'] for _ in range(len(shower['BT_X']))],
            #'chisquare': shower['chisquare']
        }
        # add very beggining of the shower
        shower_normalized['SX'] = np.append([0.], shower_normalized['SX'])
        shower_normalized['SY'] = np.append([0.], shower_normalized['SY'])
        shower_normalized['SZ'] = np.append([0.], shower_normalized['SZ'])
        
        shower_normalized['TX'] = np.append([0.], shower_normalized['TX'])
        shower_normalized['TY'] = np.append([0.], shower_normalized['TY'])
        shower_normalized['ele_P'] = np.append(shower_normalized['ele_P'], [shower_normalized['ele_P'][-1]])
        
        shower_normalized['SX'] -= shower_normalized['SZ'] * shower['ele_sx']
        shower_normalized['SY'] -= shower_normalized['SZ'] * shower['ele_sy']
        showers.append(shower_normalized)
    return showers

In [4]:
showers = normalize_showers(pmc=pmc)

100%|██████████| 9106/9106 [00:41<00:00, 221.11it/s]


In [5]:
showers = [shower for shower in showers if len(shower['TX']) > 70]
showers = [shower for shower in showers if len(shower['TX']) < 3000]

In [6]:
len(showers)

8033

In [7]:
df = pd.DataFrame(showers)

In [8]:
df.to_pickle('./data/showers.pkl')

In [9]:
showers[0].keys()

dict_keys(['SX', 'SY', 'SZ', 'TX', 'TY', 'ele_P'])

In [10]:
# len(showers[0]['SX']), len(showers[0]['SY']), len(showers[0]['SZ']), len(showers[0]['TX']), len(showers[0]['TY']), len(showers[0]['ele_P']), len(showers[0]['chisquare'])

In [11]:
df

,SX,SY,SZ,TX,TY,ele_P
0,"[0.0, 5.839430830023048, -16.20787152398225, 1...","[0.0, -17.985433881216522, 67.79949221907646, ...","[0.0, 1168.297119140625, 1169.4630126953125, 2...","[0.0, 0.004867368843406439, -0.019262431189417...","[0.0, -0.014660831540822983, 0.022187035530805...","[4.751912, 4.751912, 4.751912, 4.751912, 4.751..."
1,"[0.0, -0.1154961878200993, -7.286110487038968,...","[0.0, 2.097230334300548, 28.557897707098164, 7...","[0.0, 102.2734375, 1376.654296875, 2647.478515...","[0.0, -0.0010951315052807331, -0.0089468955993...","[0.0, 0.020027631893754005, 0.0251065474003553...","[1.2479414, 1.2479414, 1.2479414, 1.2479414, 1..."
2,"[0.0, 0.3069612562976545, 2.316498204265372, 5...","[0.0, 0.49489491642452776, 2.988925412064418, ...","[0.0, 102.39453125, 1372.13671875, 2646.5, 264...","[0.0, 0.002975787501782179, 0.0015190739650279...","[0.0, 0.004704614635556936, -0.000606656132731...","[6.491564, 6.491564, 6.491564, 6.491564, 6.491..."
3,"[0.0, 26.621750420192257, 56.109671955462545, ...","[0.0, -3.1697207751276437, -10.105108428688254...","[0.0, 1168.388671875, 2442.58203125, 2440.9829...","[0.0, 0.019973624497652054, 0.0219404138624668...","[0.0, -0.005093015264719725, -0.00309610599651...","[1.8071207, 1.8071207, 1.8071207, 1.8071207, 1..."
4,"[0.0, 1.1459594336338341, 12.985245279502124, ...","[0.0, 0.710640580726249, 8.618672659564254, 19...","[0.0, 103.005126953125, 1375.574951171875, 264...","[0.0, 0.009817498736083508, 0.0029149733018130...","[0.0, 0.006866041570901871, 0.0073389983735978...","[1.393558, 1.393558, 1.393558, 1.393558, 1.393..."
5,"[0.0, -1.2490542209416162, -15.622640912915813...","[0.0, 0.3265619672893081, 5.269950924994191, 1...","[0.0, 102.458984375, 1376.478515625, 2649.5488...","[0.0, -0.012103008106350899, -0.01588637568056...","[0.0, 0.003158319042995572, 0.0041714254766702...","[6.629736, 6.629736, 6.629736, 6.629736, 6.629..."
6,"[0.0, -0.8090329583501443, -11.47472320671659,...","[0.0, 0.7627113344678946, 11.593852424677607, ...","[0.0, 103.6640625, 1375.0703125, 2649.9453125,...","[0.0, -0.007679278030991554, -0.01256840582937...","[0.0, 0.007359364535659552, 0.0090945810079574...","[3.78361, 3.78361, 3.78361, 3.78361, 3.78361, ..."
7,"[0.0, -31.613888845662586, -57.356626820983365...","[0.0, -0.42135642320499755, -10.14111230190610...","[0.0, 1170.279296875, 2443.30078125, 3714.9453...","[0.0, -0.009577298536896706, -0.00087651633657...","[0.0, -0.006192599423229694, -0.01820842921733...","[0.8721446, 0.8721446, 0.8721446, 0.8721446, 0..."
8,"[0.0, -1.1456338084026356, -13.515102651210327...","[0.0, -0.4364222615840845, -2.6563349102507345...","[0.0, 102.193359375, 1374.458984375, 2648.9667...","[0.0, -0.01118378434330225, -0.012898478657007...","[0.0, -0.004203940276056528, -0.00082774477778...","[2.9655237, 2.9655237, 2.9655237, 2.9655237, 2..."
9,"[0.0, 1.2541289023938589, 12.67985631688498, 2...","[0.0, 2.0795444620016497, 24.006206534220837, ...","[0.0, 103.32421875, 1376.609375, 2649.39453125...","[0.0, 0.011900763027369976, 0.0134257096797227...","[0.0, 0.01978662982583046, 0.01288614980876445...","[1.5514607, 1.5514607, 1.5514607, 1.5514607, 1..."
